In [1]:
from cosmikyu import visualization as covis
from cosmikyu import gan, config, datasets, transforms
import numpy as np
import os
import torchvision.transforms as pytransforms
#from torchvision import datasets
from torchvision.utils import save_image
import matplotlib.pyplot as plt
import torch
import mlflow
import torchsummary
from orphics import maps
from pixell import enplot, enmap

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
data_dir = config.default_data_dir
sehgal_dir = os.path.join(data_dir, 'sehgal')
cuda = True
compt_idxes = [0]#,1,2,3,4]
shape = (len(compt_idxes),128,128)
#shape = (1, 256, 256)
latent_dim = 64
sample_interval = 200
save_interval = 1
batch_size = 256
nepochs=2
norm_info_file = "/home/dwhan89/workspace/cosmikyu/data/sehgal/normalization_info_fullv3.npz"

_, wcs = maps.rect_geometry(width_arcmin=64., px_res_arcmin=0.5)

/home/dwhan89/.miniconda3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [3]:
# Configure data loader
os.makedirs(data_dir, exist_ok=True)
os.makedirs(sehgal_dir, exist_ok=True)
SDN = transforms.SehgalDataNormalizer(norm_info_file, zfact=4)
SC = transforms.SehgalSubcomponets(compt_idxes)
SUBS = transforms.UnBlockShape(shape)
RF = transforms.RandomFlips(p_v=0.5, p_h=0.5)
SDS_train = datasets.SehgalDataSet(sehgal_dir, data_type="testv3", transforms=[SDN, RF, SC], dummy_label=True)

#enplot.pshow(enmap.enmap(SDS_train[0], wcs))
dataloader = torch.utils.data.DataLoader(
    SDS_train,
    batch_size=batch_size,
    shuffle=True,
)

/home/dwhan89/.miniconda3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [4]:
COSMOGAN_WGAP = gan.COSMOGAN_WGP("sehgal_cosmoganwgpv3", shape, latent_dim, cuda=cuda, ngpu=4)
mlflow.set_experiment(COSMOGAN_WGAP.identifier)
with mlflow.start_run(experiment_id=COSMOGAN_WGAP.experiment.experiment_id) as mlflow_run:
    torch.cuda.empty_cache()
    COSMOGAN_WGAP.train(
        dataloader,
        nepochs=nepochs,
        ncritics=5,
        sample_interval=sample_interval,
        save_interval=save_interval,
        load_states=True,
        save_states=True,
        verbose=True,
        mlflow_run=mlflow_run,
        lr=2e-04,
        betas=(0.5, 0.999),
        lambda_gp=10.
    )


loading saved states 
failed to load saved states
saving states _0
[Epoch 0/2] [Batch 0/118] [D loss: 9.691091] [G loss: 0.069002]
[Epoch 0/2] [Batch 5/118] [D loss: 1.258224] [G loss: 7.962967]
[Epoch 0/2] [Batch 10/118] [D loss: -18.068144] [G loss: 15.143276]
[Epoch 0/2] [Batch 15/118] [D loss: -27.405598] [G loss: 11.840870]
[Epoch 0/2] [Batch 20/118] [D loss: -30.031334] [G loss: -2.861681]
[Epoch 0/2] [Batch 25/118] [D loss: -38.956486] [G loss: -2.083562]
[Epoch 0/2] [Batch 30/118] [D loss: -39.216759] [G loss: 12.190394]
[Epoch 0/2] [Batch 35/118] [D loss: -37.839333] [G loss: 16.398941]
[Epoch 0/2] [Batch 40/118] [D loss: -37.521881] [G loss: 13.663698]
[Epoch 0/2] [Batch 45/118] [D loss: -36.576408] [G loss: 2.040830]
[Epoch 0/2] [Batch 50/118] [D loss: -36.568169] [G loss: -9.365112]
[Epoch 0/2] [Batch 55/118] [D loss: -36.363953] [G loss: -18.450888]
[Epoch 0/2] [Batch 60/118] [D loss: -38.329052] [G loss: -33.054813]
[Epoch 0/2] [Batch 65/118] [D loss: -38.041367] [G loss:

In [9]:
COSMOGAN = gan.COSMOGAN("sehgal_cosmogan", shape, latent_dim, cuda=cuda, nconv_fcdis=64, ngpu=4)
mlflow.set_experiment(COSMOGAN.identifier)          
with mlflow.start_run(experiment_id=COSMOGAN.experiment.experiment_id) as mlflow_run:
    torch.cuda.empty_cache()
    COSMOGAN.train(
        dataloader,
        nepochs=nepochs,
        ncritics=1,
        sample_interval=sample_interval,
        save_interval=save_interval,
        load_states=True,
        save_states=True,
        verbose=True,
        mlflow_run=mlflow_run,
        lr=2e-04,
        betas=(0.5, 0.999)
    )

NameError: name 'gan' is not defined

In [4]:
WGAN_GP = gan.WGAN_GP("sehgal_wgan_gp", shape, latent_dim, p_fliplabel=0.01, cuda=cuda, ngpu=4)
mlflow.set_experiment(WGAN_GP.identifier)
with mlflow.start_run(experiment_id=WGAN_GP.experiment.experiment_id) as mlflow_run:
    torch.cuda.empty_cache()
    WGAN_GP.train(
        dataloader,
        nepochs=nepochs,
        ncritics=5,
        sample_interval=sample_interval,
        save_interval=save_interval,
        load_states=True,
        save_states=True,
        verbose=True,
        mlflow_run=mlflow_run,
        lr=2e-04,
        betas=(0.5, 0.999),
        lambda_gp=10,
    )

loading saved states
failed to load saved states
[Epoch 0/2] [Batch 0/782] [D loss: 9.425166] [G loss: 0.477536]
saving states
[Epoch 0/2] [Batch 1/782] [D loss: -1967.577271] [G loss: -24.084686]
[Epoch 0/2] [Batch 2/782] [D loss: -7624.934082] [G loss: -147.843201]
[Epoch 0/2] [Batch 3/782] [D loss: -18027.080078] [G loss: -651.585876]
[Epoch 0/2] [Batch 4/782] [D loss: -33443.070312] [G loss: -2314.534912]
[Epoch 0/2] [Batch 5/782] [D loss: -51354.609375] [G loss: -6002.318359]
[Epoch 0/2] [Batch 6/782] [D loss: -71186.335938] [G loss: -10608.253906]
[Epoch 0/2] [Batch 7/782] [D loss: -91638.570312] [G loss: -13889.488281]
[Epoch 0/2] [Batch 8/782] [D loss: -103577.054688] [G loss: -15213.735352]
[Epoch 0/2] [Batch 9/782] [D loss: -102160.531250] [G loss: -15305.935547]
[Epoch 0/2] [Batch 10/782] [D loss: -101209.531250] [G loss: -14895.144531]
[Epoch 0/2] [Batch 11/782] [D loss: -105000.171875] [G loss: -14445.365234]
[Epoch 0/2] [Batch 12/782] [D loss: -105017.781250] [G loss: -14

[Epoch 0/2] [Batch 108/782] [D loss: -103720.000000] [G loss: -14728.732422]
[Epoch 0/2] [Batch 109/782] [D loss: -104101.984375] [G loss: -14960.045898]
[Epoch 0/2] [Batch 110/782] [D loss: -105426.093750] [G loss: -15155.130859]
[Epoch 0/2] [Batch 111/782] [D loss: -104824.515625] [G loss: -14788.333984]
[Epoch 0/2] [Batch 112/782] [D loss: -105270.812500] [G loss: -15101.629883]
[Epoch 0/2] [Batch 113/782] [D loss: -104917.531250] [G loss: -14978.191406]
[Epoch 0/2] [Batch 114/782] [D loss: -103643.453125] [G loss: -14668.988281]
[Epoch 0/2] [Batch 115/782] [D loss: -105388.109375] [G loss: -15202.030273]
[Epoch 0/2] [Batch 116/782] [D loss: -105031.992188] [G loss: -14873.798828]
[Epoch 0/2] [Batch 117/782] [D loss: -104367.859375] [G loss: -14836.804688]
[Epoch 0/2] [Batch 118/782] [D loss: -105327.656250] [G loss: -15196.566406]
[Epoch 0/2] [Batch 119/782] [D loss: -102216.656250] [G loss: -14314.955078]
[Epoch 0/2] [Batch 120/782] [D loss: -105578.132812] [G loss: -15453.474609]

[Epoch 0/2] [Batch 215/782] [D loss: -104408.437500] [G loss: -15447.058594]
[Epoch 0/2] [Batch 216/782] [D loss: -105750.265625] [G loss: -14609.261719]
[Epoch 0/2] [Batch 217/782] [D loss: -104904.406250] [G loss: -14993.843750]
[Epoch 0/2] [Batch 218/782] [D loss: -104094.921875] [G loss: -14690.384766]
[Epoch 0/2] [Batch 219/782] [D loss: -107085.031250] [G loss: -15593.631836]
[Epoch 0/2] [Batch 220/782] [D loss: -105142.750000] [G loss: -14119.686523]
[Epoch 0/2] [Batch 221/782] [D loss: -103292.890625] [G loss: -15481.070312]
[Epoch 0/2] [Batch 222/782] [D loss: -106182.765625] [G loss: -14916.398438]
[Epoch 0/2] [Batch 223/782] [D loss: -104183.531250] [G loss: -14499.426758]
[Epoch 0/2] [Batch 224/782] [D loss: -106385.828125] [G loss: -15711.740234]
[Epoch 0/2] [Batch 225/782] [D loss: -107195.078125] [G loss: -14327.558594]
[Epoch 0/2] [Batch 226/782] [D loss: -103183.328125] [G loss: -15040.777344]
[Epoch 0/2] [Batch 227/782] [D loss: -106237.062500] [G loss: -15516.451172]

KeyboardInterrupt: 